In [90]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from itertools import permutations, product
import functools
from sklearn.metrics import accuracy_score, confusion_matrix
import tqdm
from numba import njit,jit
import numpy as np
from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


In [2]:
df = pd.read_csv('virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

# make both have the same amount of samples..?

thresh = len(df)
# df2.drop(df2.index[thresh:len(df2.index)], inplace=True)

dfother = pd.DataFrame()
it = len(df)+1
kmer = 4

print(len(df))
print(len(df2))
# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']

    it+=1


mergedDf = pd.concat([df, dfother], axis=0)


1236
8192


In [3]:
# print(df.shape)
# print(df2.shape)

print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [4]:
kmer = 4
s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

del s
print(permset)
print(len(permset))


{'gggg', 'ccgc', 'cgcc', 'ctgt', 'actg', 'acca', 'atca', 'ggca', 'aatg', 'tcgc', 'gtcg', 'gttt', 'aaca', 'cagg', 'gtct', 'ttgc', 'aacg', 'ctac', 'ttag', 'taat', 'atac', 'agtt', 'agcc', 'tact', 'agat', 'agta', 'cttc', 'gtta', 'cacc', 'gatg', 'tgtc', 'ctct', 'aacc', 'ttaa', 'acct', 'aatt', 'cgtc', 'tttg', 'gaat', 'aaaa', 'ggct', 'tagg', 'tcag', 'gtga', 'caca', 'cccg', 'tata', 'tacc', 'aggc', 'gttg', 'gctc', 'gaac', 'acta', 'aagt', 'gacc', 'cttt', 'ccct', 'cggt', 'gttc', 'catg', 'cacg', 'ctag', 'tggg', 'accc', 'tggc', 'gaca', 'gagt', 'gcca', 'tcaa', 'tcga', 'aata', 'gagc', 'tatt', 'gata', 'cggg', 'gcgg', 'cctc', 'aaag', 'aagg', 'attt', 'gcta', 'tgtt', 'ccta', 'tgaa', 'tctt', 'taca', 'acgc', 'gatc', 'cact', 'agct', 'atga', 'tttc', 'tatc', 'cgtt', 'acat', 'tccg', 'tgcg', 'gtgg', 'cgac', 'cgcg', 'caaa', 'cgga', 'gcga', 'gtaa', 'ttct', 'gcgt', 'cgtg', 'ggcg', 'atgg', 'cccc', 'ttac', 'gaga', 'agac', 'ggaa', 'ggtg', 'agcg', 'acgt', 'acga', 'agaa', 'agtg', 'caag', 'gagg', 'gcat', 'ggag', 'ttgt',

In [5]:
def ComputeKmerVector(sequence,k,KmerCount):
    n = len(sequence)
    # find number of occurrences of certain kmers, then 
    for j in range(0,n-k+1):
        kmer = sequence[j:(j+k)]
        if kmer in KmerCount:
            KmerCount[kmer] += 1
        else:
            continue

    KmerVec = OrderedDict()

    
    for m in KmerCount:
        KmerVec[m] = float(KmerCount[m])/float(n-k+1)
    Vec = list(KmerVec.values())
    
    return Vec
def queryKmer(name,kmer):
    
    KmerCount = OrderedDict()
    s = itertools.product('acgt',repeat = kmer)
    sl = list(s)
    for i in range(0,len(sl)):
        a0 = str(sl[i])
        a1 = a0.replace("(","")
        a2 = a1.replace(")","")
        a3 = a2.replace(",","")
        a4 = a3.replace("'","")
        a4 = a4.replace(" ", "")
        KmerCount[a4] = 0
    zero = [0 for _ in range(len(sl))]
    
    total_lst = list()
    for line in name:
        tt = line.strip('\n').split(' ')
        Seq = tt[1].upper()
        KmerCount = OrderedDict(zip(KmerCount.keys(),zero))
        temp = " ".join(map(str,ComputeKmerVector(Seq,kmer,KmerCount)))
        temp = tt[0]+' '+temp
        IVL = temp.split(' ')
        total_lst.append(IVL)
    ID_VEC_LABEL = pd.DataFrame(total_lst)
    return ID_VEC_LABEL

In [6]:
dna = mergedDf['DNA Sequence']
Y = mergedDf['isZoonotic']

kmer = 4

def resetkmerdict()->OrderedDict:
    kmerdict = OrderedDict()
    for i in permset:
        kmerdict[i]=0
    return kmerdict

def assign_kmers_to_dict(row):
    kmerdict=resetkmerdict()
    st = row[2]
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict


asdf=0
l = []

for row in tqdm.tqdm(mergedDf.itertuples()):
    l.append(assign_kmers_to_dict(row))

finalkmerdict=pd.DataFrame(l)
print(finalkmerdict)

print("finished")

9428it [01:20, 116.87it/s]


      gggg  ccgc  cgcc  ctgt  actg  acca  atca  ggca  aatg  tcgc  ...  gcac  \
0       19     4     6    40    42    45    23    35    49     3  ...    27   
1       26    15    14    35    40    53    20    40    55    11  ...    40   
2       21    56    45    45    46   100    54    70    49    31  ...    65   
3       29     9    15    52    51    37    12    55    43    10  ...    42   
4       33    21    25    31    31    51    38    29    46     8  ...    22   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9423     8     7     5     8     9    13    12    18    16     4  ...     9   
9424    81    81    81   161   142    95   164   141   224    76  ...    93   
9425    21     9    15    23    33    43    31    32    35     9  ...    25   
9426    73    15    12    50    69    90    61    76    57    12  ...    30   
9427    87    26    23    35    29    79    92    53    41    22  ...    46   

      ggat  cgta  tatg  atgc  gact  tttt  cgat  aag

In [7]:
# mergedDf.drop(mergedDf.index[100:mergedDf.shape[0]], inplace=True)

for i in range(3, mergedDf.shape[1]-1):
    for j in range(0, 100):
        print(mergedDf.head().loc[j])
        str(mergedDf.head().loc[j][mergedDf.columns[i]]) != "nan" and print(mergedDf.head().loc[0][mergedDf.columns[i]])
# print(len(mergedDf))


In [8]:
mergedDf.fillna(0, inplace=True)

X = finalkmerdict
Y = mergedDf['isZoonotic']

print(X)
print(Y)

      gggg  ccgc  cgcc  ctgt  actg  acca  atca  ggca  aatg  tcgc  ...  gcac  \
0       19     4     6    40    42    45    23    35    49     3  ...    27   
1       26    15    14    35    40    53    20    40    55    11  ...    40   
2       21    56    45    45    46   100    54    70    49    31  ...    65   
3       29     9    15    52    51    37    12    55    43    10  ...    42   
4       33    21    25    31    31    51    38    29    46     8  ...    22   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9423     8     7     5     8     9    13    12    18    16     4  ...     9   
9424    81    81    81   161   142    95   164   141   224    76  ...    93   
9425    21     9    15    23    33    43    31    32    35     9  ...    25   
9426    73    15    12    50    69    90    61    76    57    12  ...    30   
9427    87    26    23    35    29    79    92    53    41    22  ...    46   

      ggat  cgta  tatg  atgc  gact  tttt  cgat  aag

In [9]:
print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

zz = X_train.head()

In [74]:
model = LogisticRegression(
    multi_class="multinomial", max_iter=1000,
    fit_intercept=False, tol=0.001, solver='saga', random_state=42)
model.fit(X_train, y_train)

LogisticRegression(fit_intercept=False, max_iter=1000,
                   multi_class='multinomial', random_state=42, solver='saga',
                   tol=0.001)

In [75]:
# all 0s for some reason
predictions = model.predict(X_test)
asdf = [x for x in predictions if x == 1]
print(asdf)
print(accuracy_score(y_test, predictions))

[1.0]
0.8676283411115825


In [76]:
knntest = BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors = 1, n_jobs = 1), n_estimators = 10, n_jobs = 1)

knntest.fit(X_train, y_train)
# knn = KNeighborsClassifier(n_neighbors=5)

BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_jobs=1,
                                                              n_neighbors=1),
                          n_jobs=1)

In [77]:
predictions = knntest.predict(X_test)
print(predictions[:10])
print(accuracy_score(y_test, predictions))

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0.9109036911327959


In [78]:
randforest = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features="auto"))

randforest.fit(X_train, y_train)

/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features='auto'))

In [79]:
def saveModel(model, name, X_test, y_test):
    predictions = model.predict(X_test)
    currAcc = accuracy_score(y_test, predictions)
    
    print(confusion_matrix(y_test, predictions))
    
    if not path.exists(f"{name}.pkl"):
        print("does not exist")
        pickle.dump(model, open(f'{name}.pkl', 'wb'))
    else:
        pickled_model = pickle.load(open(f'{name}.pkl', 'rb'))
        picklePredictions=pickled_model.predict(X_test)
        pickleAcc=accuracy_score(y_test, picklePredictions)
        print(confusion_matrix(y_test, picklePredictions))

        print("curr", currAcc, "pickle", pickleAcc)
        if currAcc > pickleAcc:
            print("update!")
            pickle.dump(model, open(f'{name}.pkl', 'wb'))
        else:
            model=pickled_model
    return model

In [80]:
saveModel(randforest, "randforest", X_test, y_test)

predictions = randforest.predict(X_test)
# print(predictions[:100])
y_test_list = [x for x in y_test[:1000]]

# for i in range(1000):
#     predictions[i] != y_test_list[i] and print(predictions[i], "vs", y_test_list[i])

[[1923  123]
 [  49  262]]
[[1973   73]
 [  24  287]]
curr 0.9270258803563852 pickle 0.958845990666101


/var/folders/vf/6r913rfx607_3y64x5l2zw4r0000gn/T/ipykernel_95295/2629373277.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test_list = [x for x in y_test[:1000]]


In [69]:
probability_predictions = randforest.predict_proba(X_test)
preds = randforest.predict(X_test)

print(probability_predictions[:10])
print(preds[:10])
print(y_test[:10])


[[1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.7 0.3]
 [1.  0. ]
 [1.  0. ]
 [0.9 0.1]
 [0.7 0.3]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1303    0.0
6597    0.0
4096    0.0
7285    0.0
4900    0.0
6290    0.0
7831    0.0
2049    0.0
7123    0.0
7489    0.0
Name: isZoonotic, dtype: float64


/var/folders/vf/6r913rfx607_3y64x5l2zw4r0000gn/T/ipykernel_95295/132363789.py:6: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(y_test[:10])


In [84]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.55, 0.6, 0.65,0.7, 0.75, 0.8, 0.85, 1]

for learning_rate in lr_list:
    gradBoost = GradientBoostingClassifier(n_estimators=300, learning_rate=learning_rate, max_features=3, max_depth=2, random_state=0)
    gradBoost.fit(X_train, y_train)

    testingAcc = accuracy_score(y_test, gradBoost.predict(X_test))
    trainingAcc = accuracy_score(y_train, gradBoost.predict(X_train))
    
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(trainingAcc))
    print("Accuracy score (validation): {0:.3f}".format(testingAcc))
    # pickle.dump(gradBoost, open('gradBoost.pkl', 'wb'))
    saveModel(gradBoost, "gradBoost", X_test, y_test)



Learning rate:  0.05
Accuracy score (training): 0.949
Accuracy score (validation): 0.941
[[2034   12]
 [ 126  185]]
[[2029   17]
 [  35  276]]
curr 0.941450997030123 pickle 0.9779380568519305
Learning rate:  0.075
Accuracy score (training): 0.956
Accuracy score (validation): 0.942
[[2025   21]
 [ 115  196]]
[[2029   17]
 [  35  276]]
curr 0.9422995333050488 pickle 0.9779380568519305
Learning rate:  0.1
Accuracy score (training): 0.963
Accuracy score (validation): 0.951
[[2028   18]
 [  98  213]]
[[2029   17]
 [  35  276]]
curr 0.9507848960543063 pickle 0.9779380568519305
Learning rate:  0.25
Accuracy score (training): 0.982
Accuracy score (validation): 0.950
[[2015   31]
 [  86  225]]
[[2029   17]
 [  35  276]]
curr 0.9503606279168434 pickle 0.9779380568519305
Learning rate:  0.5
Accuracy score (training): 0.995
Accuracy score (validation): 0.950
[[2005   41]
 [  76  235]]
[[2029   17]
 [  35  276]]
curr 0.9503606279168434 pickle 0.9779380568519305
Learning rate:  0.55
Accuracy score (

In [96]:
best_grad_boost = pickle.load(open('gradBoost.pkl', 'rb'))
best_rf = pickle.load(open('randforest.pkl', 'rb'))

print("Best Gradient Boosting Accuracy: ", accuracy_score(y_test, best_grad_boost.predict(X_test)))
print("Best Random Forest Accuracy: ", accuracy_score(y_test, best_rf.predict(X_test)))

Best Gradient Boosting Accuracy:  0.9750795334040296
Best Random Forest Accuracy:  0.957051961823966


In [97]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(best_grad_boost, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: 0.942 (0.016)


In [98]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(best_rf, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be

Accuracy: 0.905 (0.014)
